In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

## Import framework main libs

In [ ]:
from tmqrfeed.manager import DataManager
from tmqrstrategy.optimizers import OptimizerBase, OptimizerGenetic

## Import strategy from module

In [ ]:
from tmqrstrategy import StrategyAlpha
import pandas as pd
import numpy as np


def CrossUp(a, b):
    """
    A crosses up B
    """
    return (a.shift(1) < b.shift(1)) & (a > b)


def CrossDown(a, b):
    """
    A crosses down B
    """
    return (a.shift(1) > b.shift(1)) & (a < b)


class Strategy_VWAP_Bands(StrategyAlpha):
    def calc_entryexit_rules(self, bands_mult_maxval, bands_mult_step, rule_index):
        try:
            px_df = self.dm.quotes()
            
            check = px_df['v']
        except Exception:
            # In case of index based quotes
            raise Exception('Use only with ContFut!')
            #px_ser = self.dm.quotes()['equity_decision']

        vwap = ((px_df.c * px_df.v).cumsum() / px_df.v.cumsum())

        vwapbands_df = pd.DataFrame()
        vwapbands_df['c'] = px_df.c

        vwapbands_df['vwap'] = vwap #= ((vwapbands_df['c'] * vwapbands_df['v']).cumsum() / vwapbands_df['v'].cumsum())

        #bands_mult_maxval = bands_mult_maxval # highest value for bands StDev multiplier
        #bands_mult_step = bands_mult_step

        vwap_exp_std = vwapbands_df['vwap'].expanding().std()

        for m in np.arange(bands_mult_step, bands_mult_maxval+bands_mult_step, bands_mult_step):
            vwapbands_df['vwap_band_+{}'.format(m)] = vwapbands_df['vwap'] + m * vwap_exp_std
            vwapbands_df['vwap_band_-{}'.format(m)] = vwapbands_df['vwap'] - m * vwap_exp_std            
        
        bands_vwap_df = vwapbands_df.filter(regex='band')
        bands_vwap_diff = bands_vwap_df.sub(vwapbands_df.c, axis=0) * -1

        bands_vwap_diff_morethan0 = bands_vwap_diff[bands_vwap_diff >= 0]
        bands_vwap_diff_lessthan0 = bands_vwap_diff[bands_vwap_diff <= 0]

        vwapbands_df['res_level'] = bands_vwap_df[bands_vwap_diff.isin(bands_vwap_diff_morethan0.min(axis=1))].sum(1)

        vwapbands_df['sup_level'] = bands_vwap_df[bands_vwap_diff.isin(bands_vwap_diff_lessthan0.max(axis=1))].sum(1)
        
        if rule_index == 0:
            entry_rule = CrossUp(px_df.h, vwapbands_df.res_level.shift(1))
            exit_rule = CrossDown(px_df.l, vwapbands_df.sup_level.shift(1))

            return entry_rule, exit_rule
        
        
        if rule_index == 1:
            entry_rule = CrossDown(px_df.l, vwapbands_df.res_level.shift(1))
            exit_rule = CrossUp(px_df.h, vwapbands_df.sup_level.shift(1))

            return entry_rule, exit_rule
        


    def calculate(self, *args):
        (direction, bands_mult_maxval, bands_mult_step, rule_index) = args

        entry_rule, exit_rule = self.calc_entryexit_rules(bands_mult_maxval, bands_mult_step, rule_index)
        return self.exposure(entry_rule, exit_rule, direction)

In [ ]:
ALPHA_CONTEXT = {
        'name': 'ES_NewFramework_VWAP_Bands_Test', # Global alpha name, which be used for load/save from DB
        'context': { # Strategy specific settings
            # These settings only applycable for alphas derived from StrategyAlpha strategy 
            # StrategyAlpha - is a classic EXO/SmartEXO based alpha
            'index_name': 'US.6C_ContFutEOD',      # Name of EXO index to trade
            'costs_per_option': 3.0,
            'costs_per_contract': 3.0,
        },
        'wfo_params': {
            'window_type': 'rolling',  # Rolling window for IIS values: rolling or expanding
            'period': 'M',  # Period of rolling window 'M' - monthly or 'W' - weekly
            'oos_periods': 2,  # Number of months is OOS period
            'iis_periods': 12,
            # Number of months in IIS rolling window (only applicable for 'window_type' == 'rolling')
        },
        'wfo_optimizer_class': OptimizerGenetic, 
        'wfo_optimizer_class_kwargs': {
            'nbest_count': 3,
            'nbest_fitness_method': 'max',
            'population_size': 10, 
            'number_generations': 3, 
            # 'rand_seed': 1, # Uncomment this parameter to make genetic results repeatable
        },
        'wfo_opt_params': [
            ('direction', [-1]),
            ('Bands Multiplier Max Value',  [10,20]),
            ('Bands Multiplier Step',  [0.25,1,2]),
            ('Rule index', [0,1]),
        ],
        'wfo_members_count': 1,
        'wfo_costs_per_contract': 0.0,
        'wfo_scoring_type': 'modsharpe'
    }

# Run the alpha

In [ ]:
# DataManager is a core class of the framework
dm = DataManager()

# Init alpha class and run
alpha = Strategy_VWAP_Bands(dm, **ALPHA_CONTEXT)    

In [ ]:
alpha.run()

# Equity

In [ ]:
alpha.stats['series']['equity'].plot();

In [ ]:
type(alpha.stats)

In [ ]:
alpha.stats['series']['costs'].cumsum().plot()